In [2]:
import numpy as np
import pandas as pd

In [3]:
dataset = pd.read_csv(r'C:\Users\admin\Desktop\species.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [4]:
X

array([['white ', 3, 'short'],
       ['green ', 2, 'tall'],
       ['green ', 3, 'short'],
       ['white ', 3, 'short'],
       ['green ', 2, 'short'],
       ['white ', 2, 'tall'],
       ['white ', 2, 'tall'],
       ['white ', 2, 'short'],
       ['green ', 3, 'tall']], dtype=object)

In [5]:
y

array(['M', 'M', 'M', 'M', 'H', 'H', 'H', 'H', 'H'], dtype=object)

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,0] = le.fit_transform(X[:,0])
X[:,2] = le.fit_transform(X[:,2])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [8]:
X

array([[1, 3, 0],
       [0, 2, 1],
       [0, 3, 0],
       [1, 3, 0],
       [0, 2, 0],
       [1, 2, 1],
       [1, 2, 1],
       [1, 2, 0],
       [0, 3, 1]], dtype=object)

In [9]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [10]:
classifier.predict(X_test)

array(['H', 'H', 'H'], dtype='<U1')

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))

[[1 0]
 [2 0]]
              precision    recall  f1-score   support

           H       0.33      1.00      0.50         1
           M       0.00      0.00      0.00         2

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
class MultiNB(object):
    
    def __init__(self):
        self.priors = None
        self.params = None
        self.unique_labels = None
    
    def fit(self, X, y, alpha=1.0):
        assert ((alpha <= 1.0) and (alpha > 0.0)), "ERROR: smoothing parameter alpha should have value within [0.0, 1.0]!"
        self.unique_labels = np.unique(y)
        self.params = np.zeros(shape = (X.shape[1], len(self.unique_labels)))
        self.priors = np.zeros(shape = (len(self.unique_labels),))
        
        for ix,label in enumerate(self.unique_labels):
            # Boolean mask for extracting training samples corresponding to label
            mask = (y == label)
            
            # Add-1 smoothing; verified numerically that probabilities column-sum to 1
            token_counts_in_label = (np.sum(X[mask, :], axis=0) + alpha)
            total_tokens_in_label = np.sum(X[mask, :]) + X.shape[1] * alpha
            self.params[:, ix] = token_counts_in_label / total_tokens_in_label
            self.priors[ix] = np.sum(mask)/len(y)
    
    def predict_log_likelihood(self, X):

        log_params = np.log(self.params)
        log_likelihoods = np.dot(X, log_params)
        return log_likelihoods
            
    def predict(self, X):
        
        log_likelihoods = self.predict_log_likelihood(X)
        index_to_label = np.argmax(log_likelihoods, axis=1)
        pred_y = np.asarray([self.unique_labels[index] for index in index_to_label])
        
        return pred_y

In [13]:
like = MultiNB()
like.fit(X_train,y_train)

In [14]:
like.predict(X_test)

array(['M', 'M', 'H'], dtype='<U1')